In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Força uso da CPU

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')  # Desabilita GPU

# Importação de bibliotecas
from models import get_model
from utils import (
    load_data,
    create_time_sequences,
    prepare_training_data,
    train_model,
    save_model,
    generate_forecast,
    save_forecasts,
)

# ============================================================================
# CONFIGURAÇÕES GLOBAIS
# ============================================================================

# Modelo a ser usado: 'RNN', 'LSTM' ou 'CNN'
MODEL_NAME = 'RNN'

# Horizonte de previsão: 3 ou 12 meses
FORECAST_HORIZON = 12

# Parâmetros fixos
LOOK_BACK = 12  # Janela temporal de 12 meses
EPOCHS = 150
BATCH_SIZE = 1

# Regiões e produtos
REGIOES = ['ilheus', 'itabuna']
PRODUTOS = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha',
            'feijao', 'leite', 'manteiga', 'oleo', 'pao', 'tomate']

2026-01-05 20:48:55.894218: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-05 20:48:55.956283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-05 20:48:56.803860: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


# Previsão preço da cesta básica

## Treinamento de modelos

### Cesta Básica

In [3]:
# Previsão da cesta básica

print(f"🚀 Iniciando treinamento de {len(REGIOES)} regiões\n")

for regiao in REGIOES:
    print(f"\n{'='*70}")
    print(f"📍 PROCESSANDO REGIÃO: {regiao.upper()}")
    print(f"{'='*70}\n")

    # Carregamento e preparação dos dados
    path = f"../data/accb_custo_total_{regiao}.xlsx"
    df = load_data(path)
    df = create_time_sequences(df, LOOK_BACK, FORECAST_HORIZON)
    X_train, y_train, X_val = prepare_training_data(df, LOOK_BACK)

    # Criação e treinamento do modelo
    model = get_model(MODEL_NAME, LOOK_BACK, FORECAST_HORIZON)
    train_model(model, X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # Salvamento do modelo
    save_model(model, MODEL_NAME, regiao, 'cesta_basica', FORECAST_HORIZON)

    # Geração e salvamento de previsões
    results = generate_forecast(model, X_val, batch_size=BATCH_SIZE)
    save_forecasts(results, MODEL_NAME, 'cesta_basica', regiao, FORECAST_HORIZON, forecast_type='cesta')

    print(f"✅ Treinado e salvo")
    print(f"📊 Previsões: {[round(r*1000, 2) for r in results]}")

print(f"\n{'='*70}")
print(f"🎉 PROCESSO CONCLUÍDO! Todos os modelos foram treinados com sucesso.")
print(f"{'='*70}")

🚀 Iniciando treinamento de 2 regiões


📍 PROCESSANDO REGIÃO: ILHEUS



2026-01-05 20:49:27.190110: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Treinado e salvo
📊 Previsões: [524.27, 532.99, 552.63, 550.17, 549.96, 551.5, 536.99, 541.5, 550.73, 522.7, 551.56, 539.44]

📍 PROCESSANDO REGIÃO: ITABUNA

✅ Treinado e salvo
📊 Previsões: [599.68, 588.88, 588.51, 596.05, 587.89, 584.6, 580.67, 580.63, 571.23, 582.25, 581.1, 590.56]

🎉 PROCESSO CONCLUÍDO! Todos os modelos foram treinados com sucesso.


### Produtos Individuais

In [ ]:
# Previsão de produtos individuais

print(f"🚀 Iniciando treinamento de {len(REGIOES)} regiões × {len(PRODUTOS)} produtos = {len(REGIOES) * len(PRODUTOS)} modelos\n")

for regiao in REGIOES:
    print(f"\n{'='*70}")
    print(f"📍 PROCESSANDO REGIÃO: {regiao.upper()}")
    print(f"{'='*70}\n")

    for produto in PRODUTOS:
        print(f"  🔄 Produto: {produto.capitalize():<12}", end=" → ")

        # Carregamento e preparação dos dados
        path = f"../data/datasets_produtos/{regiao}/{produto}_{regiao}.xlsx"
        df = load_data(path)
        df = create_time_sequences(df, LOOK_BACK, FORECAST_HORIZON)
        X_train, y_train, X_val = prepare_training_data(df, LOOK_BACK)

        # Criação e treinamento do modelo
        model = get_model(MODEL_NAME, LOOK_BACK, FORECAST_HORIZON)
        train_model(model, X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

        # Salvamento do modelo
        save_model(model, MODEL_NAME, regiao, produto, FORECAST_HORIZON, subdir='produtos')

        # Geração e salvamento de previsões
        results = generate_forecast(model, X_val, batch_size=BATCH_SIZE)
        save_forecasts(results, MODEL_NAME, produto, regiao, FORECAST_HORIZON, forecast_type='produtos')

        print(f"✅ Treinado e salvo")

print(f"\n{'='*70}")
print(f"🎉 PROCESSO CONCLUÍDO! Todos os {len(REGIOES) * len(PRODUTOS)} modelos foram treinados com sucesso.")
print(f"{'='*70}")

🚀 Iniciando treinamento de 2 regiões × 12 produtos = 24 modelos


📍 PROCESSANDO REGIÃO: ILHEUS

  🔄 Produto: Acucar       → 

2026-01-05 20:08:42.145058: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Treinado e salvo
  🔄 Produto: Arroz        → ✅ Treinado e salvo
  🔄 Produto: Banana       → ✅ Treinado e salvo
  🔄 Produto: Cafe         → ✅ Treinado e salvo
  🔄 Produto: Carne        → WARNING:tensorflow:5 out of the last 5 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7b35c032e480> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
✅ Treinado e salvo
  🔄 Produto: Farinha      → WARNING:tensorflow:6 out of the last 6 call